In [ ]:
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
from configparser import ConfigParser
import json
import os
from os.path import exists
import pickle
import numpy as np

from myfm import MyFMRegressor
# from myfm.utils.callbacks import RegressionCallback

# class MyRegressionCallback(RegressionCallback):
#     def __call__(self, i, fm, hyper, history):
#         should_stop, description = super(MyRegressionCallback, self).__call__(i, fm, hyper, history)
#         trace_result = self.result_trace[-1]
#         if len(self.result_trace) > 8:
#             for index in range(8):
#                 old_trace_result = self.result_trace[-(index + 1)]
#                 if abs(old_trace_result['rmse'] - trace_result['rmse']) > 0.0001:
#                     return (should_stop, description)
#             return (True, description)
#         return (should_stop, description)

In [ ]:
#Read config.ini file
config = ConfigParser()
config.read("config.ini")
dataset_info = config["DATASETS"]
fm_data_path = dataset_info['fm_path']
rf_data_path = dataset_info['rf_path']
train_set_filename = dataset_info['train_filename']
test_set_filename = dataset_info['test_filename']
description_filename = dataset_info['description_filename']

model_info = config["MODELS"]
model_path = model_info['model_path']
fm_filename = model_info['fm_model_filename']
rf_filename = model_info['rf_model_filename']

### FM Training and Testing

In [ ]:
train = pickle.load(open(fm_data_path + train_set_filename, 'rb'))
test = pickle.load(open(fm_data_path + test_set_filename, 'rb'))
X_train = train.drop(['rating'], axis=1)
X_test = test.drop(['rating'], axis=1)
y_train = train['rating']
y_test = test['rating']


with open(fm_data_path + description_filename) as f:
    meta_data = json.load(f)
    group_shapes = [v for k,v in meta_data.items()]

In [ ]:

# callback = MyRegressionCallback(5, X_test, y_test.values)

# create parent folder if doesn't exist
os.makedirs(model_path, exist_ok=True)

# load from pickle dump, if it exists. Otherwise train model and then save/'pickle' it
fm_model_path = model_path + fm_filename
if exists(fm_model_path):
    fm = pickle.load(open(fm_model_path, 'rb'))
else:
    fm = MyFMRegressor(rank=1).fit(X_train, y_train, n_iter=300, group_shapes=group_shapes)
    pickle.dump(fm, open(fm_model_path, 'wb'))

fm_error = metrics.mean_squared_error(y_test, fm.predict(X_test), squared=False)
print(f'FM Regression error: {fm_error}')


### RF Training and Testing

In [ ]:
train = pickle.load(open(rf_data_path + train_set_filename, 'rb'))
test = pickle.load(open(rf_data_path + test_set_filename, 'rb'))
# train = pd.read_csv(rf_data_path + train_set_filename, sep=',', encoding='latin-1', index_col=None, nrows=1000)
# test = pd.read_csv(rf_data_path + test_set_filename, sep=',', encoding='latin-1', index_col=None, nrows=1000)
X_train = train.drop(['rating'], axis=1)
X_test = test.drop(['rating'], axis=1)
y_train = train['rating']
y_test = test['rating']

In [ ]:
rf_model_path = model_path + rf_filename
if exists(rf_model_path):
    rf = pickle.load(open(rf_model_path, 'rb'))
else:
    rf = RandomForestRegressor(n_estimators = 100, random_state = 42).fit(X_train, y_train)
    pickle.dump(rf, open(rf_model_path, 'wb'))

rf_error = metrics.mean_squared_error(y_test, rf.predict(X_test), squared=False)
print(f'Random Forest error: {rf_error}')


### RF Feature Importance

In [ ]:
import matplotlib.pyplot as plt

# list of column names
feature_names = list(X_train.columns)

# extract the feature importance values
std = np.std([tree.feature_importances_ for tree in rf.estimators_], axis=0)
rf_feature_importances = pd.DataFrame(
    {"feature": feature_names, "importance": rf.feature_importances_}
)

rf_feature_importances.sort_values("importance", ascending=False,inplace=True)

# visualize the importance of each feature
fig, ax = plt.subplots(figsize=(12,6))
rf_feature_importances.plot.bar(x='feature', y='importance', yerr=std, ax=ax, legend=False)
ax.set_title("Feature importances")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()